In [39]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score
from joblib import load
from tensorflow.keras.models import load_model
from sklearn.manifold import TSNE
from matplotlib.colors import ListedColormap
import tensorflow as tf

data_path = '../Smart Attack Algorithm/data/Car Hacking Dataset/benign_data.csv'

def hex_to_bin(hex_num):
    
    binary_value = bin(int(str(hex_num), 16))[2:]
    
    return binary_value

def int_to_bin(int_num):
    
    binary_value = bin(int_num)[2:]
    
    return binary_value

hex_to_dec = lambda x: int(x, 16)
dec_to_hex = lambda x : hex(int(x))[2:]

In [2]:
def read_data(data_path):
    
    columns = ['Timestamp','ID', 'DLC', 'Payload', 'label']
    
    data = pd.read_csv(data_path)
    
    ##Replacing all NaNs with '00' 
    data = data.replace(np.NaN, '00')

    data['ID'] = data['ID'].apply(hex_to_dec)
    
    data['Payload'] = data['Payload'].str.replace(' ', '')
    data['Payload'] = data['Payload'].apply(hex_to_dec)
    
    data = data.assign(IAT=data['Timestamp'].diff().fillna(0))
    data = data.drop(columns = ['Timestamp'], axis = 1)
    
    return data

In [4]:
normal_data = read_data(data_path)
normal_data.drop(columns = ['label'], inplace = True)
normal_data.head()

,ID,DLC,Payload,IAT
0,848,8,371692544708313250,0.000000
1,704,8,1441151880758558720,0.000221
2,1072,8,0,0.000554
3,1201,8,0,0.000238
4,497,8,0,0.000248


In [5]:
# X_train, X_test = train_test_split(normal_data, test_size=0.2, random_state=42)

# scaler = StandardScaler()

scaler = load('ch_scaler.joblib')
X_train = scaler.fit_transform(normal_data)
# X_test = scaler.transform(X_test)

/Users/anwesh/opt/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
model = load_model('ae_ch.h5')
X_train_pred = model.predict(X_train)

print("Training Loss: ", mean_squared_error(X_train, X_train_pred))

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2024-03-12 00:27:12.156197: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-12 00:27:12.158147: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


    1/30903 [..............................] - ETA: 1:45:17

2024-03-12 00:27:12.919678: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-12 00:27:13.023021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


30903/30903 [==============================] - 67s 2ms/step
Training Loss:  1.0000050705461037


In [32]:
def squared_difference_error(X, X_recon):
    # Calculate squared difference error between each sample of X and X_recon
    squared_diff = np.square(X - X_recon)

    return squared_diff

training_losses = squared_difference_error(X_train, X_train_pred)

In [10]:
# strat = MirroredStrategy()

# EPOCHS = 1000
# BATCH_SIZE = 32 * strat.num_replicas_in_sync
# LOSS = 'mse'

# # Define early stopping callback

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                               patience=20)

# early_stopper = EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True)

# input_dim = X_train.shape[1]


# with strat.scope():
#     model = Sequential()

#     ##Encoder
#     model.add(Dense(input_dim, input_shape=(input_dim, ), activation='relu'))
#     model.add(Dense(3, activation='relu'))
#     model.add(Dense(2, activation='relu'))

#     ##Bottleneck
#     model.add(Dense(1, activation='relu'))

#     ##Decoder
#     model.add(Dense(2, activation='relu'))
#     model.add(Dense(3, activation='relu'))
#     model.add(Dense(input_dim))
    
 

#     model.compile(optimizer='adam', loss=LOSS)

# history = model.fit(X_train, X_train, epochs=EPOCHS, batch_size=BATCH_SIZE, 
#                     validation_data=(X_test, X_test), callbacks=[reduce_lr, early_stopper])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


2024-03-11 18:00:54.133793: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_735"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.auto

Epoch 1/1000
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 14 all-reduces with algorithm = nccl, num_packs = 1


2024-03-11 18:01:00.320482: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


12349/12361 [============================>.] - ETA: 0s - loss: 0.4805

2024-03-11 18:01:42.283882: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_15178"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.au

12361/12361 [==============================] - 53s 3ms/step - loss: 0.4804 - val_loss: 0.2734
Epoch 2/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2658 - val_loss: 0.2470
Epoch 3/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2455 - val_loss: 0.2399
Epoch 4/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2364 - val_loss: 0.2344
Epoch 5/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2340 - val_loss: 0.2308
Epoch 6/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2323 - val_loss: 0.2316
Epoch 7/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2315 - val_loss: 0.2347
Epoch 8/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2339 - val_loss: 0.2294
Epoch 9/1000
12361/12361 [==============================] - 40s 3ms/step - loss: 0.2307 - val_loss: 0.2298
Epoch 10/1000
12361/12361 [=======================

KeyboardInterrupt: 

In [8]:
def assign_labels(X, X_recon, threshold):
    # Calculate squared error between each row of X and X_recon
    squared_errors = np.mean(np.square(X - X_recon), axis=1)

    # Assign labels based on threshold
    labels = [0 if error < threshold else 1 for error in squared_errors]

    return labels

In [9]:
car_hacking_df = pd.read_csv('../Smart Attack Algorithm/data/Car Hacking Dataset/preprocessed_car_hacking.csv')

In [10]:
car_hacking_df.head()

,Timestamp,ID,DLC,Payload,IAT,label
0,1.478198e+09,0316,8,052168092121006f,0.000000,0
1,1.478198e+09,018f,8,fe5b0000003c0000,0.000209,0
2,1.478198e+09,0260,8,19212230088e6d3a,0.000228,0
3,1.478198e+09,02a0,8,64009a1d9702bd00,0.000232,0
4,1.478198e+09,0329,8,40bb7f1411200014,0.000237,0


In [11]:
car_hacking_df.drop(columns = ['Timestamp'], inplace = True)

car_hacking_df['ID'] = car_hacking_df['ID'].apply(hex_to_dec)
car_hacking_df['Payload'] = car_hacking_df['Payload'].apply(hex_to_dec)

In [12]:
car_hacking_df.columns

Index(['ID', 'DLC', 'Payload', 'IAT', 'label'], dtype='object')

In [13]:
X = car_hacking_df.drop(columns = ['label'])
y = car_hacking_df['label']

In [14]:
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [48]:
def vary_thresholds(training_errors = training_losses, start = 0.5, end = 1, step = 0.05):

    quantiles = np.arange(start, end, step)

    thresholds = [np.quantile(training_errors, q) for q in quantiles]

    X_val_recon = model.predict(X_val)

    for threshold in thresholds:

        print("=====================================")
        print(f"Threshold: {threshold}")
        
        y_val_pred = assign_labels(X_val, X_val_recon, threshold)

        print(f"Accuracy: {accuracy_score(y_val, y_val_pred)}")

        print(f"F1 Score: {f1_score(y_val, y_val_pred)}")

        print(f"Recall: {recall_score(y_val, y_val_pred)}")

        print(f"Precision: {precision_score(y_val, y_val_pred)}")

        


vary_thresholds()

57278/57278 [==============================] - 143s 2ms/step
Threshold: 0.1544637858073506
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.17013143276783887
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.2580062999329285
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.431669409703145
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.4895234007505207
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.48952362545757894
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 1.0849276809432622
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 1.17738097

In [49]:
vary_thresholds(start = 0.05, end = 0.5, step = 0.05)

57278/57278 [==============================] - 143s 2ms/step
Threshold: 0.007256824738672733
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.05709869276351965
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.05709869276351965
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.05709869276351965
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.05709869276351965
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.05709869276351965
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.09420508967874949
Accuracy: 0.16046124006688908
F1 Score: 0.276547349496378
Recall: 1.0
Precision: 0.16046124006688908
Threshold: 0.1

In [42]:
X_test_pred = model.predict(X_test)

57278/57278 [==============================] - 119s 2ms/step


In [43]:
y_test_pred = assign_labels(X_test, X_test_pred, 1.0000050705461037)

In [44]:
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")

Accuracy: 0.16008306026670507


In [46]:
print("Classification Report: ", classification_report(y_test, y_test_pred, zero_division=0))

Classification Report:                precision    recall  f1-score   support

           0       0.00      0.00      0.00   1539472
           1       0.16      1.00      0.28    293414

    accuracy                           0.16   1832886
   macro avg       0.08      0.50      0.14   1832886
weighted avg       0.03      0.16      0.04   1832886

